<a href="https://colab.research.google.com/github/hieuza/fun/blob/main/Edit_distance_%2B_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from typing import Tuple

def min_edit_distance(
    source: str, target: str, ins_cost: int = 1, del_cost: int = 1, 
    rep_cost: int = 1) -> Tuple[np.ndarray, int]:
  nrows = len(source)
  ncols = len(target) 
  D = np.zeros((nrows + 1, ncols + 1), dtype=int) 

  D[:, 0] = range(nrows + 1)
  D[0, :] = range(ncols + 1)
  for row in range(1, nrows + 1): 
    for col in range(1, ncols + 1):
      real_rep_cost = rep_cost * (source[row - 1] != target[col - 1])
      D[row,col] = min(
          D[row - 1, col] + del_cost, 
          D[row, col - 1] + ins_cost, 
          D[row - 1, col - 1] + real_rep_cost)
        
  return D, D[nrows, ncols]

In [2]:
source = 'cat'
target = 'match'
matrix, dist = min_edit_distance(source, target, 1, 1, 1)
df = pd.DataFrame(matrix, index=list('#' + source), columns=list('#' + target))
print(df)

   #  m  a  t  c  h
#  0  1  2  3  4  5
c  1  1  2  3  3  4
a  2  2  1  2  3  4
t  3  3  2  1  2  3


In [10]:
def visualize_min_edit_distance(source: str, target: str, ins_cost: int, del_cost: int, rep_cost: int):
  """Visualize the transformation from a given source to a target."""
  D, _ = min_edit_distance(source, target, ins_cost, del_cost, rep_cost)
  
  r = len(source)
  c = len(target)
  print(f'Edit distance: {D[r][c]}')
  ops = []
  while r != 0 or c != 0:
    is_ins = c > 0 and D[r, c] == D[r, c - 1] + ins_cost
    is_del = r > 0 and D[r, c] == D[r - 1, c] + del_cost
    r_cost = rep_cost * (r > 0 and c > 0 and source[r - 1] != target[c - 1])
    is_rep = r > 0 and c > 0 and D[r, c] == D[r - 1, c - 1] + r_cost
    assert is_ins or is_del or is_rep
    op = ''
    if is_ins:
      op = ('I', c-1, target[c-1])
      c -= 1
    elif is_del:
      op = ('D', r-1, source[r-1])
      r -= 1
    elif is_rep:
      if r_cost > 0:
        op = ('R', r-1, target[c-1])
      c -= 1
      r -= 1
    if op:
      ops.insert(0, op)

  edit = source[:]
  offset = 0
  steps = np.arange(len(ops)) + 1
  pre_edits = []
  op_desc = []
  edits = []
  for step, op in enumerate(ops):
    pre_edit = edit[:]
    pos, char = op[1], op[2]
    if op[0] == 'I':
      edit = edit[:pos] + char + edit[pos:]
      offset += 1
    elif op[0] == 'D':
      pos += offset
      edit = edit[:pos] + edit[pos+1:]
      offset -= 1
    elif op[0] == 'R':
      pos += offset
      edit = edit[:pos] + char + edit[pos+1:]
      
    pre_edits.append(pre_edit)
    op_desc.append(f'{op[0]},{pos},{char}')
    edits.append(edit)
    
  df = pd.DataFrame([pre_edits, op_desc, edits], columns=steps, 
                    index=['[source]', '[op]', '[target]']).transpose()
  print(df)

source = 'zz_mode'
target = 'code_y'
visualize_min_edit_distance(source, target, 1, 1, 1)


Edit distance: 6
  [source]   [op] [target]
1  zz_mode  R,0,c  cz_mode
2  cz_mode  D,1,z   c_mode
3   c_mode  D,1,_    cmode
4    cmode  D,1,m     code
5     code  I,4,_    code_
6    code_  I,5,y   code_y
